# LINEAR 10 YEARS

In [1]:
import pandas as pd
from datetime import datetime
from fbprophet import Prophet
from matplotlib import pyplot
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score

In [26]:
# load a file
year = pd.read_csv("./SN_y_tot_V2.0.csv", sep = ";", names = ["Date_in_fraction_of_year",
                                                             "sunspot_number",
                                                             
                                                             "Daily_standard_deviation",
                                                              "Number_of_observations",
                                                             "provisional_indicator"
                                                             ])

In [27]:
year.head()

,Date_in_fraction_of_year,sunspot_number,Daily_standard_deviation,Number_of_observations,provisional_indicator
0,1700.5,8.3,-1.0,-1,1
1,1701.5,18.3,-1.0,-1,1
2,1702.5,26.7,-1.0,-1,1
3,1703.5,38.3,-1.0,-1,1
4,1704.5,60.0,-1.0,-1,1


In [28]:
# Know the type of columns
year.dtypes

Date_in_fraction_of_year    float64
sunspot_number              float64
Daily_standard_deviation    float64
Number_of_observations        int64
provisional_indicator         int64
dtype: object

In [29]:
# change the type of the column Date_in_fraction_of_year in int64 for having the correct date and cast the column
year.Date_in_fraction_of_year = year.Date_in_fraction_of_year.astype(str).str.slice(stop = 4)

In [ ]:
#

In [31]:
year_f = year[["Date_in_fraction_of_year","sunspot_number"]]

year_f.columns = ['ds', 'y']  # give the name who recognized  the prophet model
year_f['ds'] = pd.to_datetime(year_f['ds']).dt.date # transform the column in the type of datetime

<ipython-input-31-d07f82c153ab>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  year_f['ds'] = pd.to_datetime(year_f['ds']).dt.date # transform the column in the type of datetime


In [32]:
# we are going to divide the data

train = year_f.drop( year_f.index[-40:])

test =   year_f.drop( year_f.index[0:281])

In [33]:
# fitting the model

m = Prophet(n_changepoints= 20,
                changepoint_range= 0.25,
                yearly_seasonality= True,
               changepoint_prior_scale= 0.9,
                seasonality_mode= "additive"
               )

ml = m.fit(train)


INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [34]:
# prediction

future = m.make_future_dataframe(periods = 10, freq = 'Y'
                                )
forecast = m.predict(future)
forecast.head()

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,1700-01-01,159.085484,-22.763530,131.381275,159.085484,159.085484,-103.481078,-103.481078,-103.481078,-103.481078,-103.481078,-103.481078,0.0,0.0,0.0,55.604407
1,1701-01-01,160.267753,-32.942886,124.543637,160.267753,160.267753,-113.259580,-113.259580,-113.259580,-113.259580,-113.259580,-113.259580,0.0,0.0,0.0,47.008173
2,1702-01-01,161.450022,-37.994650,117.532994,161.450022,161.450022,-122.138328,-122.138328,-122.138328,-122.138328,-122.138328,-122.138328,0.0,0.0,0.0,39.311694
3,1703-01-01,162.632291,-52.126152,111.547648,162.632291,162.632291,-130.103927,-130.103927,-130.103927,-130.103927,-130.103927,-130.103927,0.0,0.0,0.0,32.528364
4,1704-01-01,163.814560,-48.379154,108.677717,163.814560,163.814560,-137.144542,-137.144542,-137.144542,-137.144542,-137.144542,-137.144542,0.0,0.0,0.0,26.670018


# CROSS_VALIDATION 10 Years

In [37]:

from fbprophet.diagnostics import cross_validation
y_cv = cross_validation(ml, initial = '90000 days', period = '210 days', horizon = '365 days')


INFO:fbprophet:Making 57 forecasts with cutoffs between 1946-10-21 00:00:00 and 1979-01-01 00:00:00


  0%|          | 0/57 [00:00<?, ?it/s]

In [38]:
# calculate rmse, mae with the cross validation

from fbprophet.diagnostics import performance_metrics

yy = performance_metrics(y_cv)
yy

,horizon,mse,rmse,mae,mape,mdape,coverage
0,27 days,4553.714538,67.481216,53.992017,0.978743,0.626521,0.8
1,36 days,4895.960815,69.971143,57.931762,1.096563,0.626521,0.8
2,45 days,7587.412122,87.105753,70.857872,0.579607,0.606061,0.6
3,54 days,4788.642371,69.200017,51.683994,0.546736,0.462163,0.8
4,55 days,5079.825626,71.272895,59.219885,0.603978,0.462163,0.8
5,63 days,4766.147708,69.037292,54.026096,0.574015,0.462163,0.8
6,64 days,4649.299337,68.185771,52.866554,0.499960,0.462163,0.8
7,72 days,5028.155931,70.909491,54.221437,0.512142,0.462163,0.8
8,73 days,5455.629330,73.862232,59.727922,0.490766,0.355282,0.8
9,82 days,5291.598440,72.743374,57.139801,0.508013,0.377528,0.8


In [39]:
# calculate Rsquare

R2 = r2_score(y_cv['y'], y_cv['yhat'])
R2


-0.1297922385010164

# LINEAR 20 YEAR

In [63]:
train.iloc[0:271,:].shape

(271, 2)

In [64]:
m_20 = Prophet(n_changepoints= 15,
                changepoint_range= 0.25,
                yearly_seasonality= True,
               changepoint_prior_scale= 0.9,
                seasonality_mode= "additive"
               )

ml_20 = m_20.fit(train.iloc[0:271,:])

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [65]:
future_20 = ml_20.make_future_dataframe(periods = 20, freq = 'Y')

In [66]:
forecast_20 = ml_20.predict(future_20)
forecast_20.head()

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,1700-01-01,115.924888,-21.017571,125.181185,115.924888,115.924888,-60.515015,-60.515015,-60.515015,-60.515015,-60.515015,-60.515015,0.0,0.0,0.0,55.409873
1,1701-01-01,116.974172,-21.890741,126.419173,116.974172,116.974172,-69.060864,-69.060864,-69.060864,-69.060864,-69.060864,-69.060864,0.0,0.0,0.0,47.913308
2,1702-01-01,118.023455,-35.936398,111.596386,118.023455,118.023455,-76.973935,-76.973935,-76.973935,-76.973935,-76.973935,-76.973935,0.0,0.0,0.0,41.049520
3,1703-01-01,119.072739,-41.765859,109.654668,119.072739,119.072739,-84.242735,-84.242735,-84.242735,-84.242735,-84.242735,-84.242735,0.0,0.0,0.0,34.830003
4,1704-01-01,120.122022,-47.449994,107.050881,120.122022,120.122022,-90.856877,-90.856877,-90.856877,-90.856877,-90.856877,-90.856877,0.0,0.0,0.0,29.265145


# CROSS VALIDATION 

In [68]:
# cross validation for evaluate the model

y_cv_20 = cross_validation(ml_20, initial = '90000 days', period = '200 days', horizon = '365 days')

INFO:fbprophet:Making 42 forecasts with cutoffs between 1946-07-21 00:00:00 and 1969-01-01 00:00:00


  0%|          | 0/42 [00:00<?, ?it/s]

In [69]:
# calculate rmse, mae

yy_20 = performance_metrics(y_cv_20 )
yy_20

,horizon,mse,rmse,mae,mape,mdape,coverage
0,34 days,10153.667482,100.765408,78.960552,0.659471,0.540163,0.75
1,43 days,9858.455167,99.289754,75.953138,0.515880,0.540163,0.75
2,51 days,5307.397693,72.851889,61.180396,0.500526,0.509454,0.75
3,60 days,5533.768361,74.389303,65.523940,0.529402,0.509454,0.75
4,69 days,5104.980511,71.449146,61.357894,0.510878,0.472405,0.75
5,78 days,4760.180448,68.994061,56.042752,0.380611,0.308377,0.75
6,86 days,1047.570556,32.366195,31.281032,0.342899,0.308377,1.00
7,95 days,3610.393467,60.086550,49.355822,0.410175,0.396982,0.75
8,104 days,3307.846937,57.513885,44.947169,0.424459,0.425551,0.75
9,112 days,4386.734736,66.232430,56.800138,0.461939,0.452381,0.75


In [78]:

# calculate Rsquare
R2 = r2_score(y_cv_20['y'], y_cv_20['yhat'])
R2

-0.23681074653531153

# LINEAR 1 YEAR

In [70]:
m_1 = Prophet(n_changepoints= 35,
                changepoint_range= 0.19,
                yearly_seasonality= True,
               changepoint_prior_scale= 0.9,
                seasonality_mode= "additive")

ml_1 = m_1.fit(train)

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [71]:
future_1 = ml_1.make_future_dataframe(periods = 1, freq = 'Y')
forecast_1 = ml_1.predict(future_1)
forecast_1.head()

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,1700-01-01,169.701738,-18.206876,130.054232,169.701738,169.701738,-114.689548,-114.689548,-114.689548,-114.689548,-114.689548,-114.689548,0.0,0.0,0.0,55.012191
1,1701-01-01,170.928277,-32.737855,121.655867,170.928277,170.928277,-124.790386,-124.790386,-124.790386,-124.790386,-124.790386,-124.790386,0.0,0.0,0.0,46.137891
2,1702-01-01,172.154816,-31.996259,119.522401,172.154816,172.154816,-133.925884,-133.925884,-133.925884,-133.925884,-133.925884,-133.925884,0.0,0.0,0.0,38.228932
3,1703-01-01,173.381355,-44.815293,104.212248,173.381355,173.381355,-142.082172,-142.082172,-142.082172,-142.082172,-142.082172,-142.082172,0.0,0.0,0.0,31.299183
4,1704-01-01,174.607894,-47.304707,99.173560,174.607894,174.607894,-149.247054,-149.247054,-149.247054,-149.247054,-149.247054,-149.247054,0.0,0.0,0.0,25.360841


# Cross validation

In [73]:
# cross validation for evaluate the model

#y_cv_20 = cross_validation(ml_20, initial = '90000 days', period = '200 days', horizon = '365 days')

y_cv_1 = cross_validation(ml_1, initial = '100000 days', period = '200 days', horizon = '365 days')

INFO:fbprophet:Making 10 forecasts with cutoffs between 1974-01-27 00:00:00 and 1979-01-01 00:00:00


  0%|          | 0/10 [00:00<?, ?it/s]

In [75]:
# show the result of cross validation
y_cv_1

,ds,yhat,yhat_lower,yhat_upper,y,cutoff
0,1975-01-01,93.679305,15.180242,166.451671,22.5,1974-01-27
1,1975-01-01,93.679305,13.930437,169.076701,22.5,1974-08-15
2,1976-01-01,93.435395,23.654742,165.347058,18.4,1975-03-03
3,1976-01-01,93.435395,12.863565,167.841987,18.4,1975-09-19
4,1977-01-01,96.988412,18.813763,167.446758,39.3,1976-04-06
5,1977-01-01,96.988412,24.212572,170.708812,39.3,1976-10-23
6,1978-01-01,93.204718,18.582193,168.548861,131.0,1977-05-11
7,1978-01-01,93.204718,18.947605,169.931710,131.0,1977-11-27
8,1979-01-01,91.265371,12.680374,165.124419,220.1,1978-06-15
9,1980-01-01,93.671475,20.688065,169.572050,218.9,1979-01-01


In [76]:
# result of the evaluation of our model
yy_1 = performance_metrics(y_cv_1)
yy_1

,horizon,mse,rmse,mae,mape,mdape,coverage
0,35 days,1428.483345,37.795282,37.795282,0.288514,0.288514,1.0
1,70 days,3327.952912,57.688412,57.688412,1.467899,1.467899,1.0
2,104 days,5630.310531,75.035395,75.035395,4.078011,4.078011,1.0
3,139 days,5066.493471,71.179305,71.179305,3.163525,3.163525,1.0
4,200 days,16598.361561,128.834629,128.834629,0.585346,0.585346,0.0
5,235 days,1428.483345,37.795282,37.795282,0.288514,0.288514,1.0
6,270 days,3327.952912,57.688412,57.688412,1.467899,1.467899,1.0
7,304 days,5630.310531,75.035395,75.035395,4.078011,4.078011,0.0
8,339 days,5066.493471,71.179305,71.179305,3.163525,3.163525,1.0
9,365 days,15682.183534,125.228525,125.228525,0.572081,0.572081,0.0


In [77]:
# calculate Rsquare
R2 = r2_score(y_cv_1['y'], y_cv_1['yhat'])
R2

-0.031442633112621854